In [52]:
import spacy
import json

In [53]:
nlp = spacy.load("en_core_web_lg")

In [61]:
categories = []

with open("categories_data.json", "r") as file:
  data = json.load(file)

  for i in data:
    categories.append(i)

In [55]:
case_data = []

with open("case_data.json", "r") as file:
  data = json.load(file)

  for case in data:
    case_data.append(case)

In [62]:
print(len(categories))
# print(len(case_data))

46


In [57]:
alpha = 0.5

In [58]:
max_thread_len = max(len(case["thread"]) for case in case_data)
print(max_thread_len)

2


In [59]:
for case in case_data:
  case_description = case["case_detail"]
  case_description = nlp(case_description)

  max_score = 0
  selected_category = 0
  total_weights = 0

  for category in categories:
    category_nlp = nlp(category["name"])

    similarity = case_description.similarity(category_nlp)

    total_weights += category["semantic_weight"]

    if (similarity > max_score):
      max_score = similarity
      selected_category = category
    
  case["case_category"] = selected_category

  category_weight = selected_category["semantic_weight"] / total_weights
  
  final_score = (alpha * category_weight) + ((1 - alpha) * max_score) + (len(case["thread"]) / max_thread_len)

  case["score"] = final_score

  priority = ""

  if (final_score < (2/3)):
    priority = "low"
  elif (final_score < (4/3) and final_score > (2/3)):
    priority = "medium"
  else:
    priority = "high"

  case["priority"] = priority

In [60]:
case_data

[{'case_no': 'CASE-001',
  'case_category': {'name': 'Water Leak/Flooding', 'semantic_weight': 4.9},
  'case_detail': 'No water supply for 3 days',
  'problem_start': '2023-06-10 08:00:00',
  'location': 'Lucknow',
  'priority': 'high',
  'score': 1.3353563875391863,
  'status': 'open',
  'thread': [{'caller_name': 'Aarav Sharma',
    'caller_phone_no': '9876543210',
    'description': 'No water in Gomti Nagar since Monday',
    'location': 'Lucknow',
    'date_time': '2023-06-10T09:23:45'},
   {'caller_name': 'Priya Singh',
    'caller_phone_no': '9567843210',
    'description': 'Water supply not restored in sector 5',
    'location': 'Lucknow',
    'date_time': '2023-06-10T11:45:30'}]},
 {'case_no': 'CASE-002',
  'case_category': {'name': 'HVAC System Failure', 'semantic_weight': 2.2},
  'case_detail': 'Power outage in colony',
  'problem_start': '2023-06-11 19:30:00',
  'location': 'Kanpur',
  'priority': 'medium',
  'score': 1.282350537261114,
  'status': 'open',
  'thread': [{'cal